###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2023 Semester 1

## Assignment 1: Music genre classification with naive Bayes


**Student ID(s):**     `1268470`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [1]:
import pandas as pd
import numpy as np

def log_gaussian(param, x):
    mu = param['mu']
    sd = param['sd']
    return -np.log(sd*np.sqrt(2*np.pi))-(x - mu)**2 / 2 / sd**2

In [14]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing

def preprocess(file):
    file = pd.read_csv(file+'.csv', sep=',')
    data = file.iloc[:, 1:-1] # exclude the first column, filename, & labels
    labels = file['label'] # label column
    return data, labels

In [49]:
# This function should calculat prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def calc_prior(labels):
    prior_prob = {}
    unique_labels, label_counts = np.unique(labels, return_counts=True)
    n = len(labels)
    for i in range(len(unique_labels)):
        prior_prob[unique_labels[i]] = (label_counts[i] / n)
    return prior_prob

In [37]:
def calc_likelihoods(data, labels):
    unique_labels = np.unique(labels) # set of labels
    features_list = data.columns # the name of the features, ensuring to exclude label*
    likelihood = {}
    
    for feature in features_list:
        likelihood[feature] = {}
        for label in unique_labels:
            values = data.loc[labels==label, feature]
            likelihood[feature][label] = {'mu': np.mean(values), 'sd': np.std(values, ddof=1)}
            # find all values for a certain feature with class == label, then record the mean and standard deviation
            
    return likelihood
    

In [38]:
def train(data, labels):
    prior_prob = calc_prior(labels) # dictionary: keys the label & value the corresponding prior probability
    likelihoods = calc_likelihoods(data, labels) # dictionary: first key the feature, second key the label then the mu & sd
    return prior_prob, likelihoods

In [52]:
def calc_posterior(data, prior_prob, likelihood):
    posterior_probs = []

    for _, instance in data.iterrows():
        post_probs = {}
        for label in prior_prob:
            post_probs[label] = np.log(prior_prob[label])
            for feature in likelihood:
                    post_probs[label] += log_gaussian(likelihood[feature][label], instance[feature])
        posterior_probs.append(post_probs)

    return posterior_probs

In [57]:
# This function should predict classes for new items in a test dataset

def predict(data, prior_prob, likelihood):
    # Calculate posterior probabilities
    posterior_probs = calc_posterior(data, prior_prob, likelihood)
    # Find argmax for each instance
    predictions = [max(probs, key=probs.get) for probs in posterior_probs]
    return predictions

In [61]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate(predictions, true):
    return sum([a == b for a,b in zip(predictions, true)])/len(predictions)

In [90]:
def diagnostics(predictions, true):
    pos = 'classical'
    predictions = pd.Series(predictions)
    tp = sum(predictions[true == pos] == pos)
    tn = sum(predictions[true != pos] != pos)
    fp = sum(predictions[true != pos] == pos)
    fn = sum(predictions[true == pos] != pos)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    print("tp: {}, tn: {}, fp: {}, fn: {}".format(tp, tn, fp, fn))
    print("precision: ",precision)
    print("recall: ", recall)
    print("accuracy: ", evaluate(predictions, true))

## Task 1. Pop vs. classical music classification

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

### Q1
Compute and report the accuracy, precision, and recall of your model (treat "classical" as the "positive" class).

In [88]:
data_train, labels_train = preprocess('pop_vs_classical_train')
prior_prob, likelihood = train(data_train, labels_train)

data_test, labels_test = preprocess('pop_vs_classical_test')
predictions = predict(data_test, prior_prob, likelihood)


In [91]:
diagnostics(predictions, labels_test)

tp: 20, tn: 22, fp: 1, fn: 0
precision:  0.9523809523809523
recall:  1.0
accuracy:  0.9767441860465116


### Q2
For each of the features X below, plot the probability density functions P(X|Class = pop) and P(X|Class = classical). If you had to classify pop vs. classical music using just one of these three features, which feature would you use and why? Refer to your plots to support your answer.
- spectral centroid mean
- harmony mean
- tempo

In [93]:
features = ['spectral_centroid_mean', 'harmony_mean', 'tempo']
labels = ['pop', 'classical']

{'classical': {'mu': 1340.3810399556526, 'sd': 361.5581884807616}, 'pop': {'mu': 3066.604994628646, 'sd': 603.7650752430725}}


In [95]:
import matplotlib.pyplot as plt
from scipy.stats import norm
for feat in features:
    for lab in labels:
        mu = likelihood[feature][lab]['mu']
        sd = likelihood[feature][lab]['sd']
        x = np.linspace(mu - 3*sd, mu + 3*sd, 100)
        plt.plot(x, stats.norm.pdf(x, mu, sigma))
        plt.show()

ModuleNotFoundError: No module named 'matplotlib'

## Task 2. 10-way music genre classification

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer must be submitted separately as a PDF.

### Q3
Compare the performance of the full model to a 0R baseline and a one-attribute baseline. The one-attribute baseline should be the best possible naive Bayes model which uses only a prior and a single attribute. In your write-up, explain how you implemented the 0R and one-attribute baselines.

### Q4
Train and test your model with a range of training set sizes by setting up your own train/test splits. With each split, use cross-fold validation so you can report the performance on the entire dataset (1000 items). You may use built-in functions to set up cross-validation splits. In your write-up, evaluate how model performance changes with training set size.

### Q5
Implement a kernel density estimate (KDE) naive Bayes model and compare its performance to your Gaussian naive Bayes model. You may use built-in functions and automatic ("rule of thumb") bandwidth selectors to compute the KDE probabilities, but you should implement the naive Bayes logic yourself. You should give the parameters of the KDE implementation (namely, what bandwidth(s) you used and how they were chosen) in your write-up.

### Q6
Modify your naive Bayes model to handle missing attributes in the test data. Recall from lecture that you can handle missing attributes at test by skipping the missing attributes and computing the posterior probability from the non-missing attributes. Randomly delete some attributes from the provided test set to test how robust your model is to missing data. In your write-up, evaluate how your model's performance changes as the amount of missing data increases.